## Data Description
|Column|Description|
|------|-----------|
|PERIOD|transaction month|    
|cl_id|client id|
|MCC|vendor category code|
|channel_type |   customer acquisition channel |
|currency |currency|  
|TRDATETIME |transaction date / time  |
|amount| transaction amount |  
|trx_category| type of transaction, POS - payment via POS terminal, C2C_OUT - transfer to card (outgoing payment), C2C_IN - transfer to card (incoming payment), DEPOSIT - card replenishment in ATM, WD_ATM_PARTNER - cash withdrawal at ATMs of partners.|  
|target_flag| Whether the customer will continue to use the product after the grace period (1/0) (target) | 
|target_sum| the amount of POS transactions for three future months (target)|  


In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn import metrics

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df1 = pd.read_csv("train.csv")
tf1 = pd.read_csv("test.csv")

In [3]:
df1.head(5)

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum
0,01/10/2017,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0
1,01/10/2017,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0
2,01/12/2017,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0
3,01/10/2017,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0
4,01/10/2017,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0


In [4]:
# plt.figure(figsize=(14,12))
# sns.heatmap(df1.isnull())
df1 = df1.fillna("undefined")

# # df1.isnull().sum(axis=0)
# plt.figure(figsize=(14,12))
# sns.heatmap(df1.isnull())

In [5]:
tf1.isnull().sum(axis=0)
tf1 = tf1.fillna("undefined")

In [6]:
#ONE HOT ENCODING
# pd.get_dummies(obj_df, columns=["body_style", "drive_wheels"], prefix=["body", "drive"]).head()
df1.nunique()

PERIOD             19
cl_id            5000
MCC               344
channel_type        6
currency           59
TRDATETIME      58113
amount          85817
trx_category       10
target_flag         2
target_sum       2651
dtype: int64

In [7]:
# count=0
# for i in df1.groupby("cl_id").count().target_flag:
#     if count < i:
#         count = i
        
# count

In [8]:
# for i in df1.cl_id:
#     print(i)
#     pass
    

In [9]:
# # def month_extract(x):
#     x = str(x)
#     return(int(x.split("/")[1]))

# df1.PERIOD = df1.PERIOD.apply(month_extract)
# tf1.PERIOD = tf1.PERIOD.apply(month_extract)

In [10]:
# # df1.currency.unique()
# df1.shape

In [11]:
# df2 = pd.get_dummies(df1, columns=["PERIOD", "cl_id","MCC","channel_type","currency","trx_category"], prefix=["month", "client","vendor","channel","currency","trx_cat"])

In [12]:
# # df3 = pd.get_dummies(df1, columns=["PERIOD","MCC","channel_type","currency","trx_category"], prefix=["month","vendor","channel","currency","trx_cat"])
# df3 = df3.drop( ["cl_id","TRDATETIME","target_sum"], axis=1 )

In [13]:
# df2 = df2.drop( ["TRDATETIME","target_sum"], axis=1 )

In [14]:
# df3.columns

In [15]:
# x = df2.drop(["target_flag"],axis=1)
# y = df2.target_flag
# y = pd.get_dummies(y, columns=["target_flag"], prefix=["target"])

In [16]:
# import os
# import pandas as pd
# from sklearn.model_selection import train_test_split
# import tensorflow as tf
# import numpy as np
# from sklearn import metrics
# from keras.models import Sequential
# from keras.layers.core import Dense, Activation
# from keras.callbacks import EarlyStopping
# from keras.callbacks import ModelCheckpoint

# def to_xy(df, target):
#     result = []
#     for x in df.columns:
#         if x != target:
#             result.append(x)
#     # find out the type of the target column.  Is it really this hard? :(
#     target_type = df[target].dtypes
#     target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
#     # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
#     if target_type in (np.int64, np.int32):
#         # Classification
#         dummies = pd.get_dummies(df[target])
#         return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
#     else:
#         # Regression
#         return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

# # Create x & y for training

# # Create the x-side (feature vectors) of the training

# x, y = to_xy(df3,'target_flag')
# # Split into train/test
# x_train, x_test, y_train, y_test = train_test_split(    
#     x, y, test_size=0.25, random_state=42) 

# # Build network
# model = Sequential()
# model.add(Dense(5, input_dim=x.shape[1], activation='relu'))
# model.add(Dense(2, activation='relu'))
# model.add(Dense(y.shape[1],activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam')
# monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
# checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=0, save_best_only=True) # save best model
# print(model.summary())
# model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor,checkpointer],verbose=2,epochs=1000)
# model.load_weights('best_weights.hdf5') # load weights from best model

# # Measure accuracy
# pred = model.predict(x_test)
# pred = np.argmax(pred,axis=1)
# y_compare = np.argmax(y_test,axis=1)
# score = metrics.accuracy_score(y_compare, pred)
# print("Final accuracy: {}".format(score))


In [17]:
temp2 = pd.get_dummies(df1, columns=["MCC","channel_type","currency","trx_category"], prefix=["vendor","channel","currency","trx_cat"])
temp2 = temp2.drop( ["PERIOD","TRDATETIME","target_sum"], axis=1 )

In [18]:
temp2.columns

Index(['cl_id', 'amount', 'target_flag', 'vendor_742', 'vendor_763',
       'vendor_780', 'vendor_1520', 'vendor_1711', 'vendor_1731',
       'vendor_1750',
       ...
       'trx_cat_BACK_TRX', 'trx_cat_C2C_IN', 'trx_cat_C2C_OUT',
       'trx_cat_CASH_ADV', 'trx_cat_CAT', 'trx_cat_DEPOSIT', 'trx_cat_POS',
       'trx_cat_WD_ATM_OTHER', 'trx_cat_WD_ATM_PARTNER', 'trx_cat_WD_ATM_ROS'],
      dtype='object', length=422)

In [47]:
x1=[]
y1=[]
# First loop iterate over every user
for i in temp2.groupby(["cl_id"]):
    x2 = []
    tempdf = pd.DataFrame(i[1])
    if tempdf["target_flag"].sum() != 0:
        y1.append(np.array([0,1],dtype=np.float32))
    else:
        y1.append(np.array([1,0],dtype=np.float32))
    tempdf = tempdf.drop( ["cl_id","target_flag"], axis=1 )
    #Second loop iterate over every transaction
    for index, row in tempdf.iterrows():
        x3 = []
        #third loop iterate over every feature
        for i in row:
            x3.append(i)
        x2.append(np.array(x3,dtype=np.float32))
    if len(x2) < 784:
        for j in range(784-len(x2)):
            x2.append(np.zeros(420))
    x1.append(np.array(x2))
x1 = np.array(x1)
y1 = np.array(y1)

In [48]:
x1

array([[[ 5023.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        [20000.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        [  767.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        ...,
        [    0.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        [    0.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        [    0.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ]],

       [[  380.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        [  378.        ,     0.        ,     0.        , ...,
             0.        ,     0.        ,     0.        ],
        [  199.        , 

In [49]:
print(x1.shape,y1.shape)
X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size=0.33, random_state=42)

(5000, 784, 420) (5000, 2)


In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [65]:
print('Build model...')
model = Sequential()
model.add(LSTM(256, dropout=0.2, input_shape=(None, 420),activation='relu'))
# model.add(LSTM(128))
# model.add(Dense(200,activation = 'relu'))
model.add(Dense(2, activation='softmax'))

# try using different optimizers and different optimizer configs

model.compile(loss='categorical_crossentropy',
              optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5", verbose=1, save_best_only=True) # save best model
print('Train...')
model.fit(X_train,y_train,validation_data=[X_test,y_test],callbacks=[monitor,checkpointer],verbose=1,epochs=200,batch_size=512)
model.load_weights('best_weights.hdf5')
pred = model.predict(X_test)
predict_classes = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_compare, predict_classes)
print("Final accuracy: {}".format(score))


Build model...
Train...
Train on 3350 samples, validate on 1650 samples
Epoch 1/200
3350/3350 [==============================] - 25s 7ms/step - loss: 1.6532 - val_loss: 0.6926

Epoch 00001: val_loss improved from inf to 0.69264, saving model to best_weights.hdf5
Epoch 2/200
3350/3350 [==============================] - 23s 7ms/step - loss: 0.6926 - val_loss: 0.6918

Epoch 00002: val_loss improved from 0.69264 to 0.69181, saving model to best_weights.hdf5
Epoch 3/200
3350/3350 [==============================] - 23s 7ms/step - loss: 0.6922 - val_loss: 0.6910

Epoch 00003: val_loss improved from 0.69181 to 0.69099, saving model to best_weights.hdf5
Epoch 4/200
3350/3350 [==============================] - 23s 7ms/step - loss: 0.6916 - val_loss: 0.6903

Epoch 00004: val_loss improved from 0.69099 to 0.69030, saving model to best_weights.hdf5
Epoch 5/200
3350/3350 [==============================] - 23s 7ms/step - loss: 0.6913 - val_loss: 0.6896

Epoch 00005: val_loss improved from 0.69030 to 

In [60]:
model.load_weights('best_weights.hdf5')
pred = model.predict(X_test)
predict_classes = np.argmax(pred,axis=1)
y_compare = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_compare, predict_classes)
print("Final accuracy: {}".format(score))

Final accuracy: 0.5709090909090909


In [66]:
pred

array([[0.46310514, 0.53689486],
       [0.46310514, 0.53689486],
       [0.46310514, 0.53689486],
       ...,
       [0.46310514, 0.53689486],
       [0.46310514, 0.53689486],
       [0.46310514, 0.53689486]], dtype=float32)